---
title: "[Kaggle] - Click-Through Rate Prediciton - Azura"
date: "2025-09-22"
category: "Recommendation"
tags: ["Recommendation Study"]
excerpt: "CTR prediction modeling"
---

- [캐글 url](https://www.kaggle.com/competitions/avazu-ctr-prediction/overview)

In [1]:
import os

os.listdir("./avazu-ctr-prediction/")

['test.gz', 'train.gz', 'sampleSubmission.gz']

- train 
    - Training set. 10 days of click-through data, ordered chronologically. 
    - Non-clicks and clicks are subsampled according to different strategies.
- test 
    - Test set. 1 day of ads to for testing your model predictions. 
- sampleSubmission.csv 
    - Sample submission file in the correct format, corresponds to the All-0.5 Benchmark.

In [2]:
import pandas as pd

train_df = pd.read_csv("./avazu-ctr-prediction/train.gz") # 읽는데 엄청 오래 걸리네 40초 정도

In [3]:
y_col = "click" # binary, 0/1 for non-click/click

In [4]:
train_df.sample(10)

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
4763561,1.121357e+19,0,14102205,1005,0,6ec06dbd,d262cf1e,f66779e6,ecad2386,7801e8d9,...,1,0,21790,320,50,2513,3,35,100194,68
39982692,5.261055e+18,0,14103019,1005,1,856e6d3f,58a89a43,f028772b,ecad2386,7801e8d9,...,1,0,23929,320,50,2743,0,163,100141,17
1274707,1.550341e+19,0,14102106,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,2,15699,320,50,1722,0,35,-1,79
18943465,1.022859e+19,0,14102516,1005,1,d9750ee7,98572c79,f028772b,ecad2386,7801e8d9,...,1,0,18092,320,50,2060,3,39,-1,23
19572031,1.337161e+19,1,14102519,1005,0,85f751fd,c4e18dd6,50e219e0,e2fcccd2,5c5a694b,...,1,0,20632,320,50,2374,3,39,-1,23
23956216,1.635862e+19,0,14102701,1002,0,cc26ad12,660b8f68,50e219e0,ecad2386,7801e8d9,...,0,0,17566,320,50,479,3,39,100076,23
780362,1.775009e+19,0,14102104,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15708,320,50,1722,0,35,100083,79
8741795,4.024636e+18,0,14102218,1005,0,7294ea0f,863fa89d,3e814130,ecad2386,7801e8d9,...,1,0,20366,320,50,2333,0,39,-1,157
29765191,1.236848e+19,0,14102813,1002,0,9e8e8d09,7f629af8,50e219e0,ecad2386,7801e8d9,...,0,0,23204,300,50,2672,0,35,-1,221
16202832,1.556553e+19,0,14102417,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15708,320,50,1722,0,35,100083,79


In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40428967 entries, 0 to 40428966
Data columns (total 24 columns):
 #   Column            Dtype  
---  ------            -----  
 0   id                float64
 1   click             int64  
 2   hour              int64  
 3   C1                int64  
 4   banner_pos        int64  
 5   site_id           object 
 6   site_domain       object 
 7   site_category     object 
 8   app_id            object 
 9   app_domain        object 
 10  app_category      object 
 11  device_id         object 
 12  device_ip         object 
 13  device_model      object 
 14  device_type       int64  
 15  device_conn_type  int64  
 16  C14               int64  
 17  C15               int64  
 18  C16               int64  
 19  C17               int64  
 20  C18               int64  
 21  C19               int64  
 22  C20               int64  
 23  C21               int64  
dtypes: float64(1), int64(14), object(9)
memory usage: 7.2+ GB


In [6]:
train_df.isna().mean()

id                  0.0
click               0.0
hour                0.0
C1                  0.0
banner_pos          0.0
site_id             0.0
site_domain         0.0
site_category       0.0
app_id              0.0
app_domain          0.0
app_category        0.0
device_id           0.0
device_ip           0.0
device_model        0.0
device_type         0.0
device_conn_type    0.0
C14                 0.0
C15                 0.0
C16                 0.0
C17                 0.0
C18                 0.0
C19                 0.0
C20                 0.0
C21                 0.0
dtype: float64

Azura CTR 예측 문제의 데이터 필드들에 대해 설명드리겠습니다. 이는 광고 클릭률(Click-Through Rate) 예측을 위한 데이터셋으로, 각 필드는 광고 노출과 클릭에 영향을 미치는 다양한 요소들을 나타냅니다.

**기본 식별자 및 타겟 변수:**
- `id`: 광고 식별자로, 각 광고 노출을 고유하게 구분하는 ID입니다.
- `click`: 타겟 변수로, 0은 클릭하지 않음, 1은 클릭함을 의미합니다.
- `hour`: 시간 정보로 YYMMDDHH 형식(예: 14091123은 2014년 9월 11일 23:00 UTC)입니다. 사용자의 시간대별 행동 패턴을 파악하는 데 중요합니다.

**광고 관련 피처:**
- `banner_pos`: 배너 광고의 위치를 나타냅니다. 웹페이지나 앱 내에서 광고가 표시되는 위치(상단, 하단, 사이드바 등)에 따라 클릭률이 달라질 수 있습니다.
- `C1`: 익명화된 범주형 변수로, 광고의 특성이나 유형을 나타낼 가능성이 높습니다.

**사이트/웹 관련 피처:**
- `site_id`: 광고가 노출된 웹사이트의 고유 식별자입니다.
- `site_domain`: 웹사이트의 도메인명으로, 신뢰성이나 인지도가 클릭률에 영향을 줄 수 있습니다.
- `site_category`: 웹사이트의 카테고리(뉴스, 쇼핑, 엔터테인먼트 등)로, 사용자의 관심사와 광고의 관련성에 따라 클릭률이 달라집니다.

**앱 관련 피처:**
- `app_id`: 광고가 노출된 모바일 앱의 고유 식별자입니다.
- `app_domain`: 앱의 도메인 정보입니다.
- `app_category`: 앱의 카테고리(게임, 소셜, 유틸리티 등)로, 앱의 성격이 광고 클릭률에 영향을 미칩니다.

**디바이스 관련 피처:**
- `device_id`: 사용자의 디바이스 고유 식별자입니다.
- `device_ip`: 디바이스의 IP 주소로, 지역 정보나 네트워크 환경을 추정할 수 있습니다.
- `device_model`: 디바이스 모델명으로, 디바이스 성능이나 사용자 세그먼트를 파악하는 데 도움이 됩니다.
- `device_type`: 디바이스 유형(스마트폰, 태블릿, 데스크톱 등)으로, 사용 환경에 따라 광고 상호작용 패턴이 다릅니다.
- `device_conn_type`: 네트워크 연결 유형(WiFi, 4G, 5G 등)으로, 연결 속도나 안정성이 광고 로딩과 상호작용에 영향을 줄 수 있습니다.

**추가 범주형 변수:**
- `C14-C21`: 익명화된 범주형 변수들로, 사용자 행동, 광고 특성, 또는 기타 컨텍스트 정보를 나타낼 수 있습니다. 각각의 구체적인 의미는 데이터 제공자가 보안상 공개하지 않았지만, 모델링 과정에서 중요한 예측 변수로 활용될 수 있습니다.

이러한 피처들은 모두 광고 클릭률에 영향을 미칠 수 있는 다양한 요인들을 나타내며, 머신러닝 모델에서 사용자의 클릭 행동을 예측하는 데 중요한 역할을 합니다. 특히 시간, 디바이스, 사이트/앱 특성 등의 조합을 통해 사용자의 클릭 확률을 예측하는 것이 이 문제의 핵심입니다.

In [7]:
train_df['banner_pos'].value_counts()

banner_pos
0    29109590
1    11247282
7       43577
2       13001
4        7704
5        5778
3        2035
Name: count, dtype: int64

In [8]:
train_df['C1'].value_counts()

C1
1005    37140632
1002     2220812
1010      903457
1012      113512
1007       35304
1001        9463
1008        5787
Name: count, dtype: int64

In [32]:
from sklearn.preprocessing import LabelEncoder

df = train_df.copy()
df = df[["click", "banner_pos", "C1", "hour"]]

# hour 분해
# if 'hour' in df.columns:
#     df['day'] = df['hour'] // 100
#     df['hour_of_day'] = df['hour'] % 100
#     df = df.drop('hour', axis=1)

# string -> numeric
# for col in df.select_dtypes(include=['object']).columns:
#     df[col] = LabelEncoder().fit_transform(df[col].astype(str))

In [ ]:
# 그냥 이진 분류 모델 만들고 성능 확인해보기

tr_df = df[~df['hour'].astype(str).str.contains('141030', na=False)] # 9일
ts_df = df[df['hour'].astype(str).str.contains('141030', na=False)] # 1일

# 올바른 방법
X_tr = tr_df.drop(['click', 'hour'], axis=1)  # 피처들
y_tr = tr_df['click']  # 타겟 변수

X_ts = ts_df.drop(['click', 'hour'], axis=1)  # 피처들
y_ts = ts_df['click']  # 타겟 변수

In [38]:
from sklearn.linear_model import LogisticRegression

# lg_reg = LogisticRegression(solver="liblinear", random_state=42)
lg_reg = LogisticRegression(
    solver='saga',
    class_weight='balanced',  # 자동으로 클래스 비율에 반비례하는 가중치
    C=0.1,
    random_state=42
)
# 수동 가중치 설정
# model = LogisticRegression(
#     solver='saga',
#     class_weight={0: 1, 1: 5},  # 클릭(1)에 5배 가중치
#     C=0.1,
#     random_state=42
# )

lg_reg.fit(X_tr, y_tr)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,0.1
,fit_intercept,True
,intercept_scaling,1
,class_weight,'balanced'
,random_state,42
,solver,'saga'
,max_iter,100
,multi_class,'deprecated'


In [28]:
y_tr.value_counts(normalize=True)

click
0    0.830194
1    0.169806
Name: proportion, dtype: float64

In [39]:
# 예측 확률과 예측 클래스
y_pred_proba = lg_reg.predict_proba(X_ts)[:, 1]
y_pred = lg_reg.predict(X_ts)
print(y_pred.mean())

0.23072322940038464


/Users/user/miniconda3/lib/python3.12/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/user/miniconda3/lib/python3.12/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/user/miniconda3/lib/python3.12/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/Users/user/miniconda3/lib/python3.12/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/user/miniconda3/lib/python3.12/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/user/miniconda3/lib/python3.12/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


In [40]:
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score, recall_score, precision_score, f1_score

# 메트릭 계산
print("=== Baseline Logistic Regression Results ===")
print(f"Log Loss: {log_loss(y_ts, y_pred_proba):.4f}")
print(f"ROC AUC: {roc_auc_score(y_ts, y_pred_proba):.4f}")
print(f"Accuracy: {accuracy_score(y_ts, y_pred):.4f}")
print(f"Precision: {precision_score(y_ts, y_pred):.4f}")
print(f"Recall: {recall_score(y_ts, y_pred):.4f}")
print(f"F1 Score: {f1_score(y_ts, y_pred):.4f}")

# CTR 관련 메트릭
actual_ctr = y_ts.mean()
predicted_ctr = y_pred_proba.mean()
print(f"\nActual CTR: {actual_ctr:.4f}")
print(f"Predicted CTR: {predicted_ctr:.4f}")
print(f"CTR Bias: {predicted_ctr - actual_ctr:.4f}")

=== Baseline Logistic Regression Results ===
Log Loss: 0.6904
ROC AUC: 0.5335
Accuracy: 0.6920
Precision: 0.1994
Recall: 0.2718
F1 Score: 0.2301

Actual CTR: 0.1693
Predicted CTR: 0.4983
CTR Bias: 0.3290


# 2. Catbost

In [44]:
train_df.columns

Index(['id', 'click', 'hour', 'C1', 'banner_pos', 'site_id', 'site_domain',
       'site_category', 'app_id', 'app_domain', 'app_category', 'device_id',
       'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14',
       'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21'],
      dtype='object')

In [53]:
tr_df = train_df[~train_df['hour'].astype(str).str.contains('141030', na=False)] # 9일
ts_df = train_df[train_df['hour'].astype(str).str.contains('141030', na=False)] # 1일

feature_cols = ['C1', 'banner_pos', 'site_id', 'site_domain',
       'site_category', 'app_id', 'app_domain', 'app_category']

X_tr = tr_df[feature_cols]  # 피처들
y_tr = tr_df['click']  # 타겟 변수

X_ts = ts_df[feature_cols]  # 피처들
y_ts = ts_df['click']  # 타겟 변수

In [60]:
from catboost import CatBoostClassifier

# 최종 권장 코드
model = CatBoostClassifier(
    iterations=300,
    learning_rate=0.1,
    depth=4,
    class_weights=[1, 5],  # 클릭에 5배 가중치
    random_seed=42,
    verbose=100
)

In [61]:
# 훈련 (범주형 컬럼 자동 처리)
model.fit(
    X_tr, y_tr,
    cat_features=feature_cols,
)

0:	learn: 0.6795121	total: 13.8s	remaining: 1h 8m 32s


: 

In [56]:
# 예측
y_pred_proba = model.predict_proba(X_ts)[:, 1]

# 성능 평가
from sklearn.metrics import log_loss, roc_auc_score
print(f"Log Loss: {log_loss(y_ts, y_pred_proba):.4f}")
print(f"ROC AUC: {roc_auc_score(y_ts, y_pred_proba):.4f}")

Log Loss: 0.6357
ROC AUC: 0.7282


In [58]:
print(y_pred.mean())

0.23072322940038464


In [59]:
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score, recall_score, precision_score, f1_score

# 메트릭 계산
print("=== Baseline Logistic Regression Results ===")
print(f"Log Loss: {log_loss(y_ts, y_pred_proba):.4f}")
print(f"ROC AUC: {roc_auc_score(y_ts, y_pred_proba):.4f}")
print(f"Accuracy: {accuracy_score(y_ts, y_pred):.4f}")
print(f"Precision: {precision_score(y_ts, y_pred):.4f}")
print(f"Recall: {recall_score(y_ts, y_pred):.4f}")
print(f"F1 Score: {f1_score(y_ts, y_pred):.4f}")

# CTR 관련 메트릭
actual_ctr = y_ts.mean()
predicted_ctr = y_pred_proba.mean()
print(f"\nActual CTR: {actual_ctr:.4f}")
print(f"Predicted CTR: {predicted_ctr:.4f}")
print(f"CTR Bias: {predicted_ctr - actual_ctr:.4f}")

=== Baseline Logistic Regression Results ===
Log Loss: 0.6357
ROC AUC: 0.7282
Accuracy: 0.6920
Precision: 0.1994
Recall: 0.2718
F1 Score: 0.2301

Actual CTR: 0.1693
Predicted CTR: 0.4680
CTR Bias: 0.2987
